## Jupyter notebook to create graphs using data from different linux monitoring tools

In [1]:
import matplotlib.pyplot as plt
import fileinput
import numpy as np
import re

In [2]:
def read_file(file_name):
    lines = list()
    for line in fileinput.input(file_name):
        lines.append(line)
    return lines

In [3]:
def get_tests(lines):
    test = list()
    tests = list()
    
    for i in lines:
        if len(re.findall("#test",i)) > 0:
            tests.append(test)
            test = list()
        else:
            test.append(i)
            
    tests.append(test)
    
    return tests[:4]

In [4]:
def get_memory_values(lines):
    kbmemused = list()
    memusedpercent = list()
    for i in lines:
        _,_,_,kbmemused_,memusedpercent_,_,_,_,_ = i.rstrip().split()
        kbmemused.append(float(kbmemused_))
        memusedpercent.append(float(memusedpercent_))
        
    return (kbmemused,memusedpercent)

In [5]:
def get_memory(lines):    
    kbs = list()
    percents = list()
    for i in get_tests(lines):
        kbmemused,memusedpercent = get_memory_values(i)
        kbmemused,memusedpercent = (max(kbmemused),max(memusedpercent))
        kbs.append(kbmemused)
        percents.append(memusedpercent)
        
    kbs = sorted(kbs)
    percents = sorted(percents)
    
    
    return (sum(kbs[:3])/3,sum(percents[:3])/3)

In [6]:
def get_cpu_times(lines,repetitions):
    idle = list()
    iowait = list()
    user = list()
    system = list()
    maximum = list()
    for i in lines:
        ##print(i.rstrip().split())
        _,_,_,user_,_,system_,iowait_,_,idle_= i.rstrip().split()
        iowait.append(float(iowait_))
        idle.append(float(idle_))
        user.append(float(user_))
        system.append(float(system_))
        maximum.append(100 - float(idle_))
    return (user,system,idle,iowait,maximum)

In [7]:
def get_cpu(lines,repetitions):
    totals = list()
    idles = list()
    iowaits = list()
    users = list()
    systems = list()
    maximums = list()
    for i in get_tests(lines):
        user,system,idle,iowait,maximum = get_cpu_times(i,repetitions)
        total,user,system,idle,iowait,maximum = (float(len(i))/repetitions,sum(user)*0.01/repetitions,
                                         sum(system)*0.01/repetitions,sum(idle)*0.01/repetitions,sum(iowait)*0.01/repetitions,
                                        max(maximum))
        totals.append(total)
        idles.append(idle)
        iowaits.append(iowait)
        users.append(user)
        systems.append(system)
        maximums.append(maximum)
    
    ##print(totals)
    totals = sorted(totals)
    idles = sorted(idles)
    iowaits = sorted(iowaits)
    users = sorted(users)
    systems = sorted(systems)
    maximums= sorted(maximums)
    
    return (sum(totals[:3])/3,sum(users[:3])/3,sum(systems[:3])/3,sum(idles[:3])/3,sum(iowaits[:3])/3,sum(maximums[:3])/3)
    

In [8]:
def get_io_values(lines):
    tps = list()
    breads = list()
    bwrtns = list()
    for i in lines:
        _,_,tps_,_,_,breads_,bwrtns_ = i.rstrip().split()
        tps.append(float(tps_))
        breads.append(float(breads_))
        bwrtns.append(float(bwrtns_))
        
    return (tps,breads,bwrtns)

In [21]:
def get_io(lines):
    network = list()
    disk = list()
    for i in get_tests(lines):
        tps,breads,bwrtns = get_io_values(i)
        tps,bwreads  = (sum(tps),sum(breads) + sum(bwrtns) )
        network.append(tps)
        disk.append(bwreads)
    
    network = sorted(network)
    disk = sorted(disk)
    
    return (sum(network[:3])/3,sum(disk[:3])/3)

In [10]:
time,user,system,idle,iowait,maximum = (0,1,2,3,4,5)
graphs = ["time","user","system","idle","iowait","maximum"]

def cpu_times(file,repetitions):
    lines = read_file(file)[3:]
    return get_cpu(lines,repetitions)

def create_graphs_cpu(labels,files,repetitions,saveloc,graph=time,ybottom=0):
    values = list()
    print(len(labels))
    print(len(repetitions))
    for x in range(len(labels)):
        values.append(cpu_times(files[x],repetitions[x]))  
    
    x = np.arange(len(labels))
    
    values = [x[graph] for x in values]

    print(values)

    fig, ax = plt.subplots(figsize=(17,5))
   
    plt.bar(x,values)
    if(ybottom != 0):
        ax.set_ylim(bottom=ybottom)
    plt.xticks(x,tuple(labels))
    ##plt.show()
    plt.savefig(saveloc)
    plt.close()
    

In [11]:
def create_graphs_cpu_part1(directory,test,size,saveloc,repetitions=1,graph=time,ybottom=0):
    compilers = [('gnuO3','5.3.0'),('gnuO2','5.3.0'),('gnuO1','5.3.0'),
    ('gnuO3','6.1.0'),('gnuO2','6.1.0'),('gnuO1','6.1.0'),
    ('gnuO3','7.2.0'),('gnuO2','7.2.0'),('gnuO1','7.2.0'),
    ('intelO3','2019'),('intelO2','2019'),('intelO1','2019')]
    
    repetitions = [repetitions for x in range(len(compilers))]
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_cpu(labels,files,repetitions,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["B","C"]
sizes_omp = ["C","D"]
repetitions_ser = [1,1]
repetitions_omp = [10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
        for size in range(len(sizes_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_cpu_part1(f"esc-tests/part1/{machines[machine]}/omp/cpu/",
                                        tests_omp[test],sizes_omp[size],f"images/part1/{machines[machine]}/omp/cpu/",
                                        repetitions_omp[size],graph)
                
                create_graphs_cpu_part1(f"esc-tests/part1/{machines[machine]}/ser/cpu/",
                                        tests_ser[test],sizes_ser[size],f"images/part1/{machines[machine]}/ser/cpu/",
                                        repetitions_ser[size],graph)

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/cpu/is.C.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/cpu/is.C.intelO3.2019.txt', 'esc-tests/r662/omp/cpu/is.C.intelO2.2019.txt', 'esc-tests/r662/omp/cpu/is.C.intelO1.2019.txt']
[1.7666666666666666, 1.5333333333333332, 1.7, 1.5, 1.5, 1.5, 1.7333333333333334, 1.5333333333333332, 1.5333333333333332, 2.466666666666667, 2.166666666666667, 2.433333333333333]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.

[0.0029999999999999996, 0.003966666666666666, 0.004300000000000001, 0.002933333333333333, 0.0042, 0.0042, 0.004066666666666666, 0.0025666666666666663, 0.0028666666666666667, 0.005366666666666666, 0.004333333333333333, 0.002933333333333333]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/cpu/is.D.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/cpu/is.D.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/cpu/is.D.intelO3.2019.txt', 'esc-tests/r662/omp/cpu/is.D.intelO2.2019.txt', 'esc-tests/r662/omp/cpu/is.D.intelO1.2019.txt']
[52.6666

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO2.5.3.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO1.5.3.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO3.6.1.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO2.6.1.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO1.6.1.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO3.7.2.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO2.7.2.0.txt', 'esc-tests/r662/ser/cpu/is.C.gnuO1.7.2.0.txt', 'esc-tests/r662/ser/cpu/is.C.intelO3.2019.txt', 'esc-tests/r662/ser/cpu/is.C.intelO2.2019.txt', 'esc-tests/r662/ser/cpu/is.C.intelO1.2019.txt']
[0.0069, 0.007066666666666666, 0.005733333333333333, 0.008600000000000002, 0.006866666666666667, 0.009733333333333332, 0.005566666666666667, 0.0044, 0.006999999999999999, 0.0058, 0.0056, 0.008466666666666666]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5

[8.1475, 9.450166666666666, 11.079299999999998, 8.145933333333334, 9.452399999999999, 9.4525, 6.517600000000001, 7.820333333333333, 9.126333333333333, 10.427233333333334, 6.1895, 8.7965]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/cpu/ep.C.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/cpu/ep.C.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/cpu/ep.C.intelO3.2019.txt', 'esc-tests/r662/omp/cpu/ep.C.intelO2.2019.txt', 'esc-tests/r662/omp/cpu/ep.C.intelO1.2019.txt']
[0.00124, 0.0014600000000000001, 0.00144, 0.00234333333333333

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO2.5.3.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO1.5.3.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO3.6.1.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO2.6.1.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO1.6.1.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO3.7.2.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO2.7.2.0.txt', 'esc-tests/r662/ser/cpu/mg.C.gnuO1.7.2.0.txt', 'esc-tests/r662/ser/cpu/mg.C.intelO3.2019.txt', 'esc-tests/r662/ser/cpu/mg.C.intelO2.2019.txt', 'esc-tests/r662/ser/cpu/mg.C.intelO1.2019.txt']
[69.7727666666667, 69.4290666666667, 92.57330000000007, 73.33790000000005, 82.14610000000005, 83.44860000000006, 55.74546666666669, 81.48683333333338, 73.3560666666667, 104.9508666666667, 45.951600000000006, 58.00673333333336]
['gnuO3.5.3.0', 'gnuO

...

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/cpu/ep.C.gnuO3.5.3.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO2.5.3.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO1.5.3.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO3.6.1.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO2.6.1.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO1.6.1.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO3.7.2.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO2.7.2.0.txt', 'esc-tests/r641/omp/cpu/ep.C.gnuO1.7.2.0.txt', 'esc-tests/r641/omp/cpu/ep.C.intelO3.2019.txt', 'esc-tests/r641/omp/cpu/ep.C.intelO2.2019.txt', 'esc-tests/r641/omp/cpu/ep.C.intelO1.2019.txt']
[10.340806666666671, 10.361983333333333, 10.48078333333333, 10.324056666666666, 10.286256666666668, 10.438060000000002, 10.32289, 10.32409333333333, 10.744969999999997, 5.368526666666668, 5.452233333333333, 5.47552]
['gnuO3.5.3.0', 'gnuO2.5.3.0', '

[176.66666666666666, 177.66666666666666, 179.0, 180.0, 177.33333333333334, 181.0, 181.0, 180.33333333333334, 188.0, 88.0, 91.33333333333333, 92.33333333333333]
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/ser/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO2.5.3.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO1.5.3.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO3.6.1.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO2.6.1.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO1.6.1.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO3.7.2.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO2.7.2.0.txt', 'esc-tests/r641/ser/cpu/mg.C.gnuO1.7.2.0.txt', 'esc-tests/r641/ser/cpu/mg.C.intelO3.2019.txt', 'esc-tests/r641/ser/cpu/mg.C.intelO2.2019.txt', 'esc-tests/r641/ser/cpu/mg.C.intelO1.2019.txt']
[52.0, 59.0, 66.66666666666667, 54.0, 70.33333333333333, 78.33333333333333, 47.333333333

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/cpu/mg.C.gnuO3.5.3.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO2.5.3.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO1.5.3.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO3.6.1.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO2.6.1.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO1.6.1.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO3.7.2.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO2.7.2.0.txt', 'esc-tests/r641/omp/cpu/mg.C.gnuO1.7.2.0.txt', 'esc-tests/r641/omp/cpu/mg.C.intelO3.2019.txt', 'esc-tests/r641/omp/cpu/mg.C.intelO2.2019.txt', 'esc-tests/r641/omp/cpu/mg.C.intelO1.2019.txt']
[8.433333333333334, 8.533333333333333, 8.666666666666666, 8.466666666666667, 8.566666666666668, 8.666666666666666, 8.466666666666667, 8.566666666666668, 8.533333333333333, 11.066666666666668, 10.5, 11.166666666666666]
['gnuO3.5.3.0', 'gnuO2.5.3.0',

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/ser/cpu/cg.B.gnuO3.5.3.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO2.5.3.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO1.5.3.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO3.6.1.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO2.6.1.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO1.6.1.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO3.7.2.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO2.7.2.0.txt', 'esc-tests/r641/ser/cpu/cg.B.gnuO1.7.2.0.txt', 'esc-tests/r641/ser/cpu/cg.B.intelO3.2019.txt', 'esc-tests/r641/ser/cpu/cg.B.intelO2.2019.txt', 'esc-tests/r641/ser/cpu/cg.B.intelO1.2019.txt']
[0.0176, 0.027066666666666666, 0.025599999999999994, 0.0257, 0.026600000000000002, 0.027499999999999997, 0.022199999999999998, 0.018699999999999998, 0.020266666666666665, 0.12233333333333335, 0.08843333333333332, 0.13326666666666664]
['gnuO3.5.3.0'

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/cpu/mg.D.gnuO3.5.3.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO2.5.3.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO1.5.3.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO3.6.1.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO2.6.1.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO1.6.1.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO3.7.2.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO2.7.2.0.txt', 'esc-tests/r641/omp/cpu/mg.D.gnuO1.7.2.0.txt', 'esc-tests/r641/omp/cpu/mg.D.intelO3.2019.txt', 'esc-tests/r641/omp/cpu/mg.D.intelO2.2019.txt', 'esc-tests/r641/omp/cpu/mg.D.intelO1.2019.txt']
[0.0041333333333333335, 0.005833333333333334, 0.0031000000000000003, 0.0005, 0.0016333333333333332, 0.0016333333333333332, 0.0, 0.0023666666666666667, 0.0008999999999999999, 0.007766666666666667, 0.0037333333333333333, 0.0044666666666666665]
['gnuO

In [13]:
def create_graphs_cpu_part2(directory,test,sizes,saveloc,repetitions=1,graph=time,ybottom=0):
    compiler = 'gnuO3.5.3.0'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(repetitions)
    print(files)
    create_graphs_cpu(labels,files,repetitions,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["S","W","A","B","C"]
sizes_omp = ["S","W","A","B","C","D"]
repetitions_ser = [100,10,10,1,1]
repetitions_omp = [100,100,100,100,10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_cpu_part2(f"esc-tests/part2/{machines[machine]}/omp/cpu/",
                                        tests_omp[test],sizes_omp,f"images/part2/{machines[machine]}/omp/cpu/",
                                        repetitions_omp,graph)
                create_graphs_cpu_part2(f"esc-tests/part2/{machines[machine]}/ser/cpu/",
                                            tests_ser[test],sizes_ser,f"images/part2/{machines[machine]}/ser/cpu/",
                                            repetitions_ser,graph)

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r662/omp/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/is.D.gnuO3.5.3.0.txt']
6
6
[0.12, 0.14333333333333334, 0.1366666666666667, 0.4166666666666667, 1.5333333333333332, 51.0]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r662/ser/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/is.C.gnuO3.5.3.0.txt']
5
5
[0.013333333333333334, 0.16666666666666666, 1.9000000000000001, 10.0, 37.666666666666664]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r662/omp/cpu/is.S.gnuO3.5.3.0.txt', 'esc-

['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r662/ser/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/cpu/mg.C.gnuO3.5.3.0.txt']
5
5
[0.009785666666666666, 0.29308666666666666, 2.5089099999999998, 6.189633333333333, 55.08720000000002]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r662/omp/cpu/ep.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/ep.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/ep.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/ep.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/ep.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/cpu/ep.D.gnuO3.5.3.0.txt']
6
6
[5.466666666666667e-05, 4.9e-05, 2983.2379330000003, 0.0005416666666666666, 0.0021300000000000004, 0.004666666666666667]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r662/ser/cpu/mg

['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r641/omp/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/is.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/is.D.gnuO3.5.3.0.txt']
6
6
[0.09896900000000002, 0.04596633333333333, 0.1783686666666667, 0.42528666666666665, 1.71531, 55.38149999999999]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r641/ser/cpu/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/is.C.gnuO3.5.3.0.txt']
5
5
[0.00028900000000000003, 0.0041333333333333335, 0.06452666666666668, 0.2286, 0.9977333333333331]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r641/omp/cpu/is.

['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r641/ser/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/cpu/mg.C.gnuO3.5.3.0.txt']
5
5
[6.333333333333333e-06, 0.00028333333333333335, 0.00234, 0.0061333333333333335, 0.0]
['S', 'W', 'A', 'B', 'C', 'D']
[100, 100, 100, 100, 10, 1]
['esc-tests/part2/r641/omp/cpu/ep.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/ep.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/ep.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/ep.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/ep.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/omp/cpu/ep.D.gnuO3.5.3.0.txt']
6
6
[57.98666666666667, 65.58666666666667, 93.57333333333334, 100.0, 100.0, 98.35333333333334]
['S', 'W', 'A', 'B', 'C']
[100, 10, 10, 1, 1]
['esc-tests/part2/r641/ser/cpu/mg.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r641/ser/

In [14]:
kb,per = (0,1)
graphs = ["kb","per"]

def memory_values(file):
    lines = read_file(file)[3:]
    return get_memory(lines)

def create_graphs_memory(labels,files,saveloc,graph=kb,ybottom=0):
    values = list()
    for x in range(len(labels)):
        values.append(memory_values(files[x]))  
    
    x = np.arange(len(labels))
    
    values = [x[graph] for x in values]
    

    fig, ax = plt.subplots(figsize=(17,5))
    plt.bar(x,values)
    if(ybottom != 0):
        ax.set_ylim(bottom=ybottom)
    plt.xticks(x,tuple(labels))
    plt.savefig(saveloc)
    plt.close()

In [13]:
def create_graphs_memory_part1(directory,test,size,saveloc,graph=time,ybottom=0):
    compilers = [('gnuO3','5.3.0'),('gnuO2','5.3.0'),('gnuO1','5.3.0'),
    ('gnuO3','6.1.0'),('gnuO2','6.1.0'),('gnuO1','6.1.0'),
    ('gnuO3','7.2.0'),('gnuO2','7.2.0'),('gnuO1','7.2.0'),
    ('intelO3','2019'),('intelO2','2019'),('intelO1','2019')]
    
    
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_memory(labels,files,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ["is","mg","cg"]
sizes_ser = ["B","C"]
sizes_omp = ["C","D"]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
        for size in range(len(sizes_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_memory_part1(f"esc-tests/{machines[machine]}/omp/memory/",
                                        tests_omp[test],sizes_omp[size],
                                        f"images/{machines[machine]}/omp/memory/",graph)
               
                create_graphs_memory_part1(f"esc-tests/{machines[machine]}/ser/memory/",
                                        tests_ser[test],sizes_ser[size],
                                        f"images/{machines[machine]}/ser/memory/",graph)

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/memory/is.C.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/memory/is.C.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/memory/is.C.intelO3.2019.txt', 'esc-tests/r662/omp/memory/is.C.intelO2.2019.txt', 'esc-tests/r662/omp/memory/is.C.intelO1.2019.txt']
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/memory/

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/memory/mg.B.gnuO3.5.3.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO2.5.3.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO1.5.3.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO3.6.1.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO2.6.1.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO1.6.1.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO3.7.2.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO2.7.2.0.txt', 'esc-tests/r662/ser/memory/mg.B.gnuO1.7.2.0.txt', 'esc-tests/r662/ser/memory/mg.B.intelO3.2019.txt', 'esc-tests/r662/ser/memory/mg.B.intelO2.2019.txt', 'esc-tests/r662/ser/memory/mg.B.intelO1.2019.txt']
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/memory/

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/memory/mg.D.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/memory/mg.D.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/memory/mg.D.intelO3.2019.txt', 'esc-tests/r662/omp/memory/mg.D.intelO2.2019.txt', 'esc-tests/r662/omp/memory/mg.D.intelO1.2019.txt']
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/memory/

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/ser/memory/mg.B.gnuO3.5.3.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO2.5.3.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO1.5.3.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO3.6.1.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO2.6.1.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO1.6.1.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO3.7.2.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO2.7.2.0.txt', 'esc-tests/r641/ser/memory/mg.B.gnuO1.7.2.0.txt', 'esc-tests/r641/ser/memory/mg.B.intelO3.2019.txt', 'esc-tests/r641/ser/memory/mg.B.intelO2.2019.txt', 'esc-tests/r641/ser/memory/mg.B.intelO1.2019.txt']
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/memory/

['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/memory/mg.D.gnuO3.5.3.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO2.5.3.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO1.5.3.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO3.6.1.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO2.6.1.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO1.6.1.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO3.7.2.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO2.7.2.0.txt', 'esc-tests/r641/omp/memory/mg.D.gnuO1.7.2.0.txt', 'esc-tests/r641/omp/memory/mg.D.intelO3.2019.txt', 'esc-tests/r641/omp/memory/mg.D.intelO2.2019.txt', 'esc-tests/r641/omp/memory/mg.D.intelO1.2019.txt']
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/ser/memory/

In [17]:
def create_graphs_memory_part2(directory,test,sizes,saveloc,graph=time,ybottom=0):
    compiler = 'gnuO3.5.3.0'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(files)
    create_graphs_memory(labels,files,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["S","W","A","B","C"]
sizes_omp = ["S","W","A","B","C","D"]
repetitions_ser = [100,10,10,1,1]
repetitions_omp = [100,100,100,100,10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_memory_part2(f"esc-tests/part2/{machines[machine]}/omp/memory/",
                                        tests_omp[test],sizes_omp,f"images/part2/{machines[machine]}/omp/memory/",
                                        graph)
                create_graphs_memory_part2(f"esc-tests/part2/{machines[machine]}/ser/memory/",
                                            tests_ser[test],sizes_ser,f"images/part2/{machines[machine]}/ser/memory/",
                                            graph)

['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/omp/memory/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.D.gnuO3.5.3.0.txt']
['S', 'W', 'A', 'B', 'C']
['esc-tests/part2/r662/ser/memory/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/memory/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/memory/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/memory/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/memory/is.C.gnuO3.5.3.0.txt']
['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/omp/memory/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/memory/is.C.gnuO3.5.3.0.txt', 'esc-tests/pa

In [18]:
tcp,disk = (0,1)
graphs = ["tcp","disk"]

def io_values(file):
    lines = read_file(file)[3:]
    return get_io(lines)

def create_graphs_io(labels,files,saveloc,graph=kb,ybottom=0):
    values = list()
    for x in range(len(labels)):
        values.append(io_values(files[x]))  
    
    x = np.arange(len(labels))
    
    values = [x[graph] for x in values]
    

    fig, ax = plt.subplots(figsize=(17,5))
    
    plt.bar(x,values)
    if(ybottom != 0):
        ax.set_ylim(bottom=ybottom)
    plt.xticks(x,tuple(labels))
    plt.savefig(saveloc)
    plt.close()

In [15]:
def create_graphs_io_part1(directory,test,size,saveloc,graph=time,ybottom=0):
    compilers = [('gnuO3','5.3.0'),('gnuO2','5.3.0'),('gnuO1','5.3.0'),
    ('gnuO3','6.1.0'),('gnuO2','6.1.0'),('gnuO1','6.1.0'),
    ('gnuO3','7.2.0'),('gnuO2','7.2.0'),('gnuO1','7.2.0'),
    ('intelO3','2019'),('intelO2','2019'),('intelO1','2019')]
    print(directory)

    
    labels = ['.'.join(x) for x in compilers]
    files = [f'{directory}{test}.{size}.{x}.txt' for x in labels ]
    print(labels)
    print(files)
    create_graphs_io(labels,files,f'{saveloc}{graphs[graph]}.{test}.{size}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ["is","cg","mg"]
sizes_ser = ["B","C"]
sizes_omp = ["C","D"]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
        for size in range(len(sizes_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_io_part1(f"esc-tests/{machines[machine]}/omp/io/",
                                    tests_omp[test],sizes_omp[size],
                                    f"images/{machines[machine]}/omp/io/",graph)
                
                create_graphs_io_part1(f"esc-tests/{machines[machine]}/ser/io/",
                                    tests_ser[test],sizes_ser[size],
                                    f"images/{machines[machine]}/ser/io/",graph)

esc-tests/r662/omp/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/io/is.C.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/io/is.C.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/io/is.C.intelO3.2019.txt', 'esc-tests/r662/omp/io/is.C.intelO2.2019.txt', 'esc-tests/r662/omp/io/is.C.intelO1.2019.txt']
esc-tests/r662/ser/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/io/is.B.g

esc-tests/r662/ser/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/io/cg.B.gnuO3.5.3.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO2.5.3.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO1.5.3.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO3.6.1.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO2.6.1.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO1.6.1.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO3.7.2.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO2.7.2.0.txt', 'esc-tests/r662/ser/io/cg.B.gnuO1.7.2.0.txt', 'esc-tests/r662/ser/io/cg.B.intelO3.2019.txt', 'esc-tests/r662/ser/io/cg.B.intelO2.2019.txt', 'esc-tests/r662/ser/io/cg.B.intelO1.2019.txt']
esc-tests/r662/omp/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/io/ep.D.g

esc-tests/r662/omp/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/omp/io/mg.D.gnuO3.5.3.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO2.5.3.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO1.5.3.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO3.6.1.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO2.6.1.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO1.6.1.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO3.7.2.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO2.7.2.0.txt', 'esc-tests/r662/omp/io/mg.D.gnuO1.7.2.0.txt', 'esc-tests/r662/omp/io/mg.D.intelO3.2019.txt', 'esc-tests/r662/omp/io/mg.D.intelO2.2019.txt', 'esc-tests/r662/omp/io/mg.D.intelO1.2019.txt']
esc-tests/r662/ser/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r662/ser/io/mg.C.g

esc-tests/r641/ser/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/ser/io/cg.B.gnuO3.5.3.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO2.5.3.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO1.5.3.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO3.6.1.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO2.6.1.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO1.6.1.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO3.7.2.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO2.7.2.0.txt', 'esc-tests/r641/ser/io/cg.B.gnuO1.7.2.0.txt', 'esc-tests/r641/ser/io/cg.B.intelO3.2019.txt', 'esc-tests/r641/ser/io/cg.B.intelO2.2019.txt', 'esc-tests/r641/ser/io/cg.B.intelO1.2019.txt']
esc-tests/r641/omp/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/io/ep.C.g

esc-tests/r641/omp/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/omp/io/mg.D.gnuO3.5.3.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO2.5.3.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO1.5.3.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO3.6.1.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO2.6.1.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO1.6.1.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO3.7.2.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO2.7.2.0.txt', 'esc-tests/r641/omp/io/mg.D.gnuO1.7.2.0.txt', 'esc-tests/r641/omp/io/mg.D.intelO3.2019.txt', 'esc-tests/r641/omp/io/mg.D.intelO2.2019.txt', 'esc-tests/r641/omp/io/mg.D.intelO1.2019.txt']
esc-tests/r641/ser/io/
['gnuO3.5.3.0', 'gnuO2.5.3.0', 'gnuO1.5.3.0', 'gnuO3.6.1.0', 'gnuO2.6.1.0', 'gnuO1.6.1.0', 'gnuO3.7.2.0', 'gnuO2.7.2.0', 'gnuO1.7.2.0', 'intelO3.2019', 'intelO2.2019', 'intelO1.2019']
['esc-tests/r641/ser/io/mg.C.g

In [20]:
def create_graphs_io_part2(directory,test,sizes,saveloc,graph=time,ybottom=0):
    compiler = 'gnuO3.5.3.0'
    
    labels = sizes
    files = [f'{directory}{test}.{size}.{compiler}.txt' for size in sizes]
    print(labels)
    print(files)
    create_graphs_io(labels,files,f'{saveloc}{graphs[graph]}.{test}.png',graph,ybottom)

machines = ["r662","r641"]
tests_omp = ["is","ep","mg"]
tests_ser = ['is','mg','cg']
sizes_ser = ["S","W","A","B","C"]
sizes_omp = ["S","W","A","B","C","D"]
repetitions_ser = [100,10,10,1,1]
repetitions_omp = [100,100,100,100,10,1]

for machine in range(len(machines)):
    for test in range(len(tests_omp)):
            for graph in range(len(graphs)):
                
                create_graphs_io_part2(f"esc-tests/part2/{machines[machine]}/omp/io/",
                                        tests_omp[test],sizes_omp,f"images/part2/{machines[machine]}/omp/io/",
                                        graph)
                create_graphs_io_part2(f"esc-tests/part2/{machines[machine]}/ser/io/",
                                            tests_ser[test],sizes_ser,f"images/part2/{machines[machine]}/ser/io/",
                                            graph)

['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/omp/io/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.D.gnuO3.5.3.0.txt']
['S', 'W', 'A', 'B', 'C']
['esc-tests/part2/r662/ser/io/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/io/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/io/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/io/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/ser/io/is.C.gnuO3.5.3.0.txt']
['S', 'W', 'A', 'B', 'C', 'D']
['esc-tests/part2/r662/omp/io/is.S.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.W.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.A.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.B.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.C.gnuO3.5.3.0.txt', 'esc-tests/part2/r662/omp/io/is.D.gnuO3.5.3.0.txt']
['S', 'W', 'A', 'B', 'C']

In [16]:
lines_cpu_gnuO1 = read_file('tests/omp/cpu/mg_D_gnuO1.txt')
lines_cpu_gnuO2 = read_file('tests/omp/cpu/mg_D_gnuO2.txt')
lines_cpu_gnuO3 = read_file('tests/omp/cpu/mg_D_gnuO3.txt')

lines_memory_gnuO1 = read_file('tests/omp/memory/mg_D_gnuO1.txt')
lines_memory_gnuO2 = read_file('tests/omp/memory/mg_D_gnuO2.txt')
lines_memory_gnuO3 = read_file('tests/omp/memory/mg_D_gnuO3.txt')

lines_io_gnuO1 = read_file('tests/omp/io/mg_D_gnuO1.txt')
lines_io_gnuO2 = read_file('tests/omp/io/mg_D_gnuO2.txt')
lines_io_gnuO3 = read_file('tests/omp/io/mg_D_gnuO3.txt')

In [17]:
print(get_cpu(lines_cpu_gnuO1[3:],1))
print(get_cpu(lines_cpu_gnuO2[3:],1))
print(get_cpu(lines_cpu_gnuO3[3:],1))

print(get_memory(lines_memory_gnuO1[3:]))
print(get_memory(lines_memory_gnuO2[3:]))
print(get_memory(lines_memory_gnuO3[3:]))

print(get_io(lines_io_gnuO1[3:]))
print(get_io(lines_io_gnuO2[3:]))
print(get_io(lines_io_gnuO3[3:]))

(168.33333333333334, 157.60420000000002, 1.1031999999999984, 9.353533333333333, 0.0018666666666666666)
(166.33333333333334, 155.89383333333333, 1.0690666666666653, 8.7382, 0.0014666666666666667)
(166.66666666666666, 154.60136666666673, 1.084499999999997, 10.980733333333339, 0.002133333333333333)
(28805112.0, 43.6)
(29057640.0, 43.98)
(28982804.0, 43.86333333333334)
(57.1, 483.74666666666667)
(51.349999999999994, 434.5666666666666)
(55.879999999999995, 447.05)


In [18]:
linesO3 = read_file('tests/omp/cpu/ep_D_gnuO3.txt')
linesO2 = read_file('tests/omp/cpu/ep_D_gnuO2.txt')
linesO1 = read_file('tests/omp/cpu/ep_D_gnuO1.txt')


In [19]:
O3 = [len(x) for x in get_tests(linesO3[3:])]
O2 = [len(x) for x in get_tests(linesO2[3:])]
O1 = [len(x) for x in get_tests(linesO1[3:])]

print(sum(O1))
print(sum(O2))
print(sum(O3))

741
711
726
